In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.11.0'

In [2]:
class Net(nn.Module):
#定义一个叫做Net的类，继承自父类nn.Module
    def __init__(self):
        super(Net, self).__init__()
        #super实现对父类的同名函数的继承和改写
# torch.nn.Conv2d(in_channels,out_channels,kernel_size,stride = 1,padding = 0,dilation = 1,groups = 1,bias = True,padding_mode = 'zeros')
        #in_channels：输入图像的通道数(RGB为3) out_channels：输出图像的通道数，即卷积核的个数 kernel_size：卷积核的大小
        self.conv1 = nn.Conv2d(1, 6, 5) #输入灰度图、6个卷积核、卷积核大小为5*5
        self.conv2 = nn.Conv2d(6, 16, 5) #输入6通道图像、16个卷积核、卷积核大小为5*5
        #定义全连接层: y = Wx + b
        #nn.Linear(in_features,out_features,bias=True)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #全连接层输入有16*5*5数据，输出为120维数据
        self.fc2 = nn.Linear(120, 84) #全连接层输入有120维，输出为84维数据
        self.fc3 = nn.Linear(84, 10)#全连接层输入有84维，输出为10维数据

    def forward(self, x):
        #torch.nn.functional.max_pool2d(input, kernel_size, stride=None, padding=0, dilation=1, ceil_mode=False, return_indices=False)
        #F.max_pool2d：最大池化层
        #卷积、Relu激活、2*2池化层池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 池化核大小为正方形时，可以用一个数字代替
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #改变数据维度，将数据展开
        #x = x.view(-1, self.num_flat_features(x))#调用自己定义的函数展开数据
        x = x.view(-1, x.size()[1:].numel())#与函数等价，但不用定义函数，代码更简洁
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    #定义一个函数，将输入数据展开
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()#实例化类Net
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
input = torch.randn(1, 1, 32, 32) #产生一个32*32的随机数，作为灰度图像
out = net(input) #调用forward()函数计算输出
#需要注意的是，Net()类继承自nn.Module，nn.Module的初始化函数会调用forward函数

print(out)

tensor([[ 0.0081,  0.0323,  0.0311,  0.0622,  0.0549, -0.0162, -0.0463,  0.1097,
         -0.0039,  0.0569]], grad_fn=<AddmmBackward0>)


In [4]:
#先清理梯度中原来的值，在调用backward函数计算梯度。否则会在原来的值上加上新的值。
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
output = net(input)
target = torch.randn(10)  #随机十个值作为真实标签
target = target.view(1, -1)  #使target和output的shape相同，-1的作用是自动匹配维度
criterion = nn.MSELoss() #确定损失函数
loss = criterion(output, target) #计算预测output和标签target之间的损失
print(loss)

tensor(1.4093, grad_fn=<MseLossBackward0>)


反向传播

In [6]:
net.zero_grad()     # 清除梯度
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0132,  0.0040, -0.0251,  0.0121,  0.0254, -0.0058])


更新权重

In [7]:
net.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [8]:
#使用python简单实现
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [9]:
#使用torch工具包来更新权重
import torch.optim as optim

#构建优化器optimizer
#优化器详见：https://blog.csdn.net/qq_34690929/article/details/79932416
optimizer = optim.SGD(net.parameters(), lr=0.01)

#每一次训练数据,执行以下操作
optimizer.zero_grad()#清零梯度
output = net(input)#计算输出
loss = criterion(output, target)#计算loss
loss.backward()#计算梯度
optimizer.step()#更新权重